In [1]:
import guardrails as gd
import openai
from rich import print as rprint
import tiktoken
import dotenv
import os
import pandas as pd

from app.database import Database, db_config

from app.guardrails_specs.specs import rail_str

dotenv.load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPEN_API_KEY


## Loading data

In [2]:
database = Database(**db_config)

test_user_id = 3

def convert_transactions_to_dataframe(user_id: int):
    transactions = database.get_all_transactions_by_user_id(user_id=user_id)
    df = pd.DataFrame(
        [
            {
                "transaction_id": transaction.transaction_id,
                "transaction_date": transaction.transaction_date,
                "transaction_description": transaction.transaction_description,
                "credit": transaction.credit,
                "amount": transaction.debit,
                "account_id": transaction.account_id,
                "category_id": transaction.category_id,
                "user_id": transaction.user_id,
                "category": category.category_name,
            }
            for transaction, category in transactions
        ]
    )

    return df

# converting transactions to a dataframe
df = convert_transactions_to_dataframe(test_user_id)
df.head()

2023-08-17 19:31:22,338 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-08-17 19:31:22,339 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-17 19:31:22,342 INFO sqlalchemy.engine.Engine select current_schema()
2023-08-17 19:31:22,343 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-17 19:31:22,346 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-08-17 19:31:22,346 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-17 19:31:22,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-17 19:31:22,349 INFO sqlalchemy.engine.Engine SELECT llm_finances.transactions.transaction_id AS llm_finances_transactions_transaction_id, llm_finances.transactions.transaction_date AS llm_finances_transactions_transaction_date, llm_finances.transactions.transaction_description AS llm_finances_transactions_transaction_description, llm_finances.transactions.credit AS llm_finances_transactions_credit, llm_finances.transactions.debit AS llm_finances_transactions_debit, ll

/Users/fernandomeira/projects/LLM-budget-assistant/app/database.py:219: SAWarning: relationship 'User.user_accounts' will copy column users.user_id to column accounts.user_id, which conflicts with relationship(s): 'Account.users' (copies users.user_id to accounts.user_id), 'User.accounts' (copies users.user_id to accounts.user_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="accounts,users"' to the 'User.user_accounts' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operat

,transaction_id,transaction_date,transaction_description,credit,amount,account_id,category_id,user_id,category
0,237,2023-01-01,"FIVE GUYS BURGERS & FR, 780-436-9885",0.0,46.66,4,17,3,Takeout
1,234,2023-01-02,"SAVE ON FOODS #6622, EDMONTON",0.0,30.31,4,24,3,Grocery
2,236,2023-01-02,"DOLLARAMA #1101, EDMONTON",0.0,46.28,4,30,3,Home Goods
3,235,2023-01-02,"MCDONALD'S #40413, EDMONTON",0.0,8.38,4,17,3,Takeout
4,231,2023-01-06,"PC EXPRESS 1549, EDMONTON",0.0,97.90,4,24,3,Grocery


## Testing cleaning transaction descriptions

**Goal**: Transaction descriptions could be very repetitive. I want to simplify them and make them easy for the users to identify them. At the same time, once the user assigns a category to it, I want to spot all the possible variations of it and use the category already assigned automatically.

#### Testing embeddings

Good examples to try:

- 'Subway 30118, Edmonton'
- 'Subway 39345, Edmonton',
- 'Subway 71142, Edmonton'

_____

- 'SHOP.SHOPPERSDRUGMART., BRAMPTON',
- 'SHOPPERS DRUG MART #03, EDMONTON',
- 'SHOPPERS DRUG MART #24, EDMONTON'

____

- 'PETRO CANADA77958, EDMONTON', 
- 'PETROCAN, EDMONTON',

____

In [42]:
# Getting the embeddings for the transaction names

from sentence_transformers import SentenceTransformer
import tiktoken

tnames = df.loc[:, ['transaction_description']].copy().drop_duplicates()

tnames

# Hugging face embeddings
model_name = "BAAI/bge-large-en"
hf_model = SentenceTransformer(model_name)

def get_hf_embeddings(text):
    return hf_model.encode(text)

tnames['hf_embeddings'] = tnames['transaction_description'].apply(get_hf_embeddings)

# OpenAI embeddings

def get_oi_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

tnames['oi_token_count'] = tnames.transaction_description.apply(lambda x: num_tokens_from_string(x, 'cl100k_base'))

print(f"Total cost: {tnames.oi_token_count.sum() * .0001}")
tnames['ada_embedding'] = tnames.transaction_description.apply(lambda x: get_oi_embedding(x, model='text-embedding-ada-002'))

Total cost: 0.0814


In [43]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

tnames.reset_index(drop=True, inplace=True)

def get_most_similar_entry(dataframe: pd.DataFrame, embedding_col:str):
    # Convert embedding_vector to a matrix
    embedding_matrix = np.stack(dataframe[embedding_col].values)

    norms = np.linalg.norm(embedding_matrix, axis=1)

    # Check if all norms are close to 1
    if np.allclose(norms, 1, atol=1e-6):  # you can adjust the tolerance level (atol) as needed
        print(f"The embeddings are normalized. {embedding_col}")
    else:
        print(f"The embeddings are not normalized.{embedding_col}")
        embedding_matrix = normalize(embedding_matrix)

    # Compute the cosine similarity matrix
    cosine_sim_matrix = cosine_similarity(embedding_matrix)

    # Find the most similar transactions for each transaction
    most_similar_indices = np.argmax(cosine_sim_matrix - np.eye(cosine_sim_matrix.shape[0]), axis=1)

    # Add the most similar transaction descriptions to the DataFrame
    return dataframe['transaction_description'].iloc[most_similar_indices].reset_index(drop=True)

tnames['similar_hf'] = get_most_similar_entry(tnames, 'hf_embeddings')
tnames['similar_ada'] = get_most_similar_entry(tnames, 'ada_embedding')

tnames.head(3)

The embeddings are not normalized.hf_embeddings
The embeddings are normalized. ada_embedding


,transaction_description,hf_embeddings,oi_token_count,ada_embedding,similar_hf,similar_ada
0,"#026 MARK'S, EDMONTON","[-0.48328263, 0.13348997, -0.4279842, 0.304612...",8,"[-0.01735464669764042, -0.016036231070756912, ...","T&T SUPERMARKET #024, EDMONTON","REAL CANADIAN SPRST154, EDMONTON"
1,"AMBLESIDE VETERINARY H, EDMONTON","[-0.27046075, 0.17282426, -0.5382303, 0.389169...",11,"[-0.010637522675096989, -0.013460425660014153,...","PETLANDSOUTHPARK, EDMONTON","OH MY DOG SPA AND GROO, EDMONTON"
2,"AMZN Mktp CA*OI0DI4SS3, WWW.AMAZON.CA","[-0.38278243, -0.04545469, -0.40891513, 0.3338...",20,"[-0.00796479918062687, -0.02523753233253956, -...","APPLE.COM/BILL, TORONTO","APPLE.COM/BILL, TORONTO"


In [47]:
tnames.loc[:, ['transaction_description', 'similar_hf', 'similar_ada']].iloc[0:40]

,transaction_description,similar_hf,similar_ada
0,"#026 MARK'S, EDMONTON","T&T SUPERMARKET #024, EDMONTON","REAL CANADIAN SPRST154, EDMONTON"
1,"AMBLESIDE VETERINARY H, EDMONTON","PETLANDSOUTHPARK, EDMONTON","OH MY DOG SPA AND GROO, EDMONTON"
2,"AMZN Mktp CA*OI0DI4SS3, WWW.AMAZON.CA","APPLE.COM/BILL, TORONTO","APPLE.COM/BILL, TORONTO"
3,"APPLE.COM/BILL, 866-712-7753","APPLE.COM/BILL, TORONTO","APPLE.COM/BILL, TORONTO"
4,"APPLE.COM/BILL, TORONTO","APPLE.COM/BILL, 866-712-7753","APPLE.COM/BILL, 866-712-7753"
5,"BAR BURRITO, EDMONTON","TIRAMISU BISTRO, EDMONTON","GOOD BUDDY RESTAURANT, EDMONTON"
6,"BATH & BODY WORKS (CAN, EDMONTON","SQ *CHACHI'S SOUTH COM, Edmonton","IKEA EDMONTON, EDMONTON"
7,"BLOCK 1912, EDMONTON","SQ *EARLS SOUTH COMMON, Edmonton","PETROCAN, EDMONTON"
8,"CDN TIRE STORE #00304, EDMONTON","SAFEWAY #8921, EDMONTON","T&T SUPERMARKET #024, EDMONTON"
9,"CINEPLEX 8030 WEB QPS, 416-323-6600","NETFLIX.COM, 866-716-0414","NETFLIX.COM, 866-716-0414"
